## 【新装版】　畢罪の花　～ひつざいのはな～

これのキャラ解析をする下準備

In [1]:
novel_path = "【新装版】　畢罪の花　～ひつざいのはな～"

character_list = []
with open("{}/chars.txt".format(novel_path),"r") as f:
    character_list = [ name[:-2] for name in f.readlines()
                              if name.find(":") != -1]
    
## 、毎に分断して一つずつの要素に
name_list = []
for char in character_list:
    for name in char.split("、"):
        name_list.append(name)

name_list

['ノア',
 'ナビア',
 'カイン',
 'ヨシュア',
 'ライラ',
 '闇夜の天使',
 'フローラ',
 '春咲姫',
 'オリビア',
 'ウェスペルス',
 '宵の明星',
 'グレン',
 'メイガス',
 '碩賢',
 'サラ',
 'ルイーザ',
 'アロン',
 'エド',
 'リリー',
 'マルタ']

## 文章を分解

In [2]:
import glob
import os
import re
import logging

## ログレベルを DEBUG に変更
logging.basicConfig(level=logging.DEBUG)


def text_dict(novel_dir):
    texts_dict = {}
    
    ## 章名の読み込み
    chap_list = []
    if os.path.isfile("{}/chapters.txt".format(novel_dir)):##章のファイルが存在するか、確認
        with open("{}/chapters.txt".format(novel_dir),"r") as f:
            ## 改行の削除(これがあると、うまく読み取れない)
            chap_list =  [re.sub("\n", "",name) for name in f.readlines()]
            
    ## 各話名の読み込み
    title_list = []
    with open("{}/titles.txt".format(novel_dir),"r") as f:
        ## 改行の削除(これがあると、うまく読み取れない)
        title_list = [re.sub("\n","",name) for name in f.readlines()]
        
    read_index = 0##どこまで処理したかを示す
    if chap_list:
        ## 章が存在する
        for chap_name in chap_list:
            new_dict = {}
            ## 各章名を keyとしてdict生成 + titleの読み込み位置を変更
            new_dict, read_index = text_read("{}/{}".format(novel_dir, chap_name),
                                            title_list, read_index)
            texts_dict[chap_name] = new_dict 
            
    else:
        ## 章が存在しない
        texts_dict,_ = text_read(novel_dir, title_list, read_index)
        
        

    return texts_dict, chap_list, title_list
        
    
    
## 各ディレクトリ中の小説txtを読み込む
def text_read(now_dir, name_list, index):
    each_text_dict = {}
    
    filepath_names = [path for path in glob.glob("{}/*".format(now_dir))
                          if path.find("titles.txt") == -1]##章が存在しない場合、現れるため
    ##
    ##logging.debug(filepath_names)
    ##logging.debug(glob.glob("{}/*".format(now_dir)))
    
    for i in range(0,len(filepath_names)):
        text_name = name_list[index]
        #logging.debug(filepath_names)
        text_path = [path for path in filepath_names
                            if path.find(text_name) != -1][0]
        
        ## txtデータ読み込み
        with open(text_path,"r") as f:
            each_text_dict[text_name] = f.read()
        index += 1 ##indexを増やす
    
    
    return each_text_dict, index

#### \n 6以上 + ***(空白なし)を\n にする?

In [6]:
import re
import logging

## ログレベルを DEBUG に変更
logging.basicConfig(level=logging.DEBUG)


def count_names(novel_dict, chap_list, char_list):
    count_dict = {}## 返す値
    
    ## 章が存在する場合
    if chap_list:
        for chap_name in chap_list:
            chap_dict = novel_dict[chap_name]
            
            ##章毎keyに対応する dict生成
            count_dict[chap_name] = count_per_story(chap_dict, char_list)
            
    
    ## 章が存在しない場合
    else:
        count_dict = count_per_story(novel_dict, char_list)
        
        
    return count_dict


## 上の関数の子供的
def count_per_story(chap_dict, char_list):
    count_dict = {}##返す値

    ##この小説における、シーン切り替えに利用する部分(大量の改行に変換する)
    make_return = r"＊＊＊|\*\*\*"
    
    
    for title, text in chap_dict.items():
        text = re.sub(r'　| ',"",text)##空白の削除(念のため)
        text = re.sub(make_return, "\n\n\n\n\n\n", text)##シーン変化時に見分ける用
        text = re.sub(r"\n{6,}","\n\n\n\n\n\n", text) ##６回以上の出現は 6にする
                
        scene_dict = {}##各話毎key の dict生成 scene_listを入れる
        scene_list = []##一話中の各シーン(list) を辞書型(name: 回数)
                
        for scene in text.split("\n\n\n\n\n\n"):
            if not scene:
                ##文字がない -> 最後の改行に騙された部分
                continue
                
            ##シーン毎に処理
            
            name_dict = {}##scene_list に入れる辞書
            
            ## 各キャラ名でカウント
            for char in char_list:
                
                ##各シーンで 名前毎の出現回数を調べる
                name_dict[char] = len(re.findall(char, scene))
                ##print(title, char, scene_dict[i][char])##debug用
                
            ## リストに 辞書(name: 回数)を保存
            scene_list.append(name_dict)
            
        ##各話key にdictをつける
        count_dict[title] = scene_list
        
    ##logging.debug(count_dict)
    return count_dict

In [7]:
## 合体した関数
def make_count_dict(novel_path, char_list):
    texts_dict, chap_list, title_list = text_dict(novel_path)
    
    return count_names(texts_dict, chap_list, char_list)

In [8]:
import json

count_dict = make_count_dict(novel_path, name_list)

## json化する( Falseにすることで、utf-8もエスケープされない)
count_txt = json.dumps(count_dict, ensure_ascii=False)
with open("char_count.json","w") as f:
    f.write(count_txt)